# Predicting FWI using deepFWI Pre-trained models

### Introduction
The Fire Weather Index (FWI) is a meteorologically based index used worldwide to estimate fire danger. It consists of different components that account for the effects of fuel moisture and wind on fire behaviour and spread. The higher the FWI, the more favorable the meteorological conditions to trigger a wildfire. This indicator can help shape long-term tourist strategy and to plan future investments under a changing climate.

The [`esowc/wildfire-forecasting`](https://github.com/esowc/wildfire-forecasting) project intends to reproduce the Fire Forecasting capabilities of GEFF using Deep Learning and develop further improvements in accuracy, geography and time scale through inclusion of additional variables or optimisation of model architecture & hyperparameters. 

In this notebook we demonstrate the use of pre-trained Deep Learning models to perform FWI inference. The Deep Learning Model has been trained on 1 year of data and takes as input the four weather forcings used in the numerical calculation of FWI:
```
* t2     -    Temperature at 2m (K)
* tp     -    Total Precipitation accumulated over the previous 24 hours (mm)
* rh     -    Relative Humidity (%)
* wspeed -    Windspeed (m/s)
```

These input weather forcings are supplied for 4 days and the model produces FWI forecast for 14 days. There is 1 day of overlap between the 4 weather forcing variable inputs and the output FWI forecast as indicated below:
```
                                       ____________
Weather Forcings (t-3, t-2, ..., t) -> | deepFWI  |  -> deepFWI-Forecast (t, t+1, t+2, ..., t+13)
                                       ------------
```

To run this notebook, make sure you have the `wildfire-dl` `conda` environment activated. Instructions for setting up the environment are available in the project [`README`](https://github.com/esowc/wildfire-forecasting/blob/master/README.md).

In [1]:
import os
from IPython.display import Image
os.chdir('..')

### Ensure access to the data

This example will require the following dataset from gs://deepgeff-data-v0/ -
* `fwi-forcings/`: Weather Forcings for 2020-12-01 to 2020-12-04 --> 4 days --> 4 files 
* `fwi-reanalysis/`: ERA Reanalysis of FWI for 2020-12-04 to 2020-12-18 --> 14 days --> 14 files

If you wish to explore these datasets further, have a look at the EDA notebooks in `data/EDA/` in the repository.

In [2]:
# Uncomment to download a minimal sample dataset consisting of Weather Forcings
# for 2019-12-01 to 2019-12-04 --> 4 days --> 4 files
!mkdir sampledata/4_14/forcings -p
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/forcings/ECMWF_FO_2019120[1-4]*' sampledata/4_14/forcings

In [3]:
# Uncomment to download a minimal sample dataset consisting of ERA Reanalysis 
# for 2019-12-04 to 2019-12-18 --> 14 days --> 14 files
!mkdir sampledata/4_14/fwi-reanalysis -p
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/fwi-reanalysis/ECMWF_FWI_201912[0][4-9]*' sampledata/4_14/fwi-reanalysis
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/fwi-reanalysis/ECMWF_FWI_201912[1][0-8]*' sampledata/4_14/fwi-reanalysis

### Getting the pre-trained model

For this example notebook, we use the following model-

- Run ID: `1ga5loax` that has been trained for 30 epochs on 3 months of T2, TP, RH and WSpeed data for July 2019 - Sept, 2019.

If you wish to explore other models, have a look at pre-trained_models.md.

In [4]:
# Uncomment to download a sample pre-trained model checkpoint file
!mkdir samplemodel/4_14/ -p
!gsutil -q cp 'gs://deepgeff-models-v0/pre-trained_models/4_14/1ga5loax/1ga5loax_59_60.ckpt' samplemodel/4_14/

### Running the inference

For performing inference and then benchmarking the results, we use [`src/test.py`](https://github.com/esowc/wildfire-forecasting/blob/master/src/test.py) which takes as argument a dict of configuration parameters. 

Providing the configuration options,

```
* in_days        -    no of input days of weather forcings
* out_days       -    no of output days of fwi prediction
* checkpoint_file-    path to pretrained checkpoint file
* reanalysis_dir -    path to fwi-reanalysis data
* forcings_dir   -    path to fwi-forcings data
```

In [5]:
!python src/test.py -in-days 4 -out-days 14 -checkpoint-file "samplemodel/4_14/1ga5loax_59_60.ckpt" \
-forcings-dir "sampledata/4_14/forcings" -reanalysis-dir "sampledata/4_14/fwi-reanalysis"

 |       init-features -> 16                  
 |             in-days -> 4                   
 |            out-days -> 14                  
 |              epochs -> 100                 
 |       learning-rate -> 0.001               
 |          batch-size -> 1                   
 |               split -> 0.2                 
 |           use-16bit -> True                
 |                gpus -> 1                   
 |               optim -> one_cycle           
 |             dry-run -> False               
 |             find-lr -> False               
 |           search-bs -> False               
 |          case-study -> False               
 |         clip-output -> False               
 |              boxcox -> 0.1182              
 |              binned -> [0, 5.2, 11.2, 21.3, 38.0, 50]
 |       round-to-zero -> False               
 |         isolate-frp -> False               
 |          date-range -> False               
 |             cb-loss -> False               
 | 

 'mae_21.3_38.0_0': tensor(12.3167, device='cuda:0'),
 'mae_21.3_38.0_1': tensor(12.1240, device='cuda:0'),
 'mae_21.3_38.0_10': tensor(12.6965, device='cuda:0'),
 'mae_21.3_38.0_11': tensor(13.2929, device='cuda:0'),
 'mae_21.3_38.0_12': tensor(13.7497, device='cuda:0'),
 'mae_21.3_38.0_13': tensor(12.7795, device='cuda:0'),
 'mae_21.3_38.0_2': tensor(12.9138, device='cuda:0'),
 'mae_21.3_38.0_3': tensor(12.3223, device='cuda:0'),
 'mae_21.3_38.0_4': tensor(13.0505, device='cuda:0'),
 'mae_21.3_38.0_5': tensor(12.4071, device='cuda:0'),
 'mae_21.3_38.0_6': tensor(13.0381, device='cuda:0'),
 'mae_21.3_38.0_7': tensor(12.1425, device='cuda:0'),
 'mae_21.3_38.0_8': tensor(14.6393, device='cuda:0'),
 'mae_21.3_38.0_9': tensor(12.9575, device='cuda:0'),
 'mae_3': tensor(14.4046, device='cuda:0'),
 'mae_38.0_50_0': tensor(11.4498, device='cuda:0'),
 'mae_38.0_50_1': tensor(10.3648, device='cuda:0'),
 'mae_38.0_50_10': tensor(11.4736, device='cuda:0'),
 'mae_38.0_50_11': tensor(12.3376, devi

 'mse_9': tensor(125.9114, device='cuda:0'),
 'test_loss': tensor(15341.0732, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|████████████████████████████████████| 1/1 [00:06<00:00,  6.93s/it]


### Results

We demonstrate below the metrics (accuracy, mean-squared-error and mean-absolute-error) for deepFWI predicted values. These metrics are for 1 FWI prediction (for 14 days) in a global setting. The metrics are calculated against fwi-reanalysis (considered the ground truth). The grouped bar plots indicate the metrics for deepFWI while the line plots are used for fwi-forecast. The FWI values are binned as per the EFFIS categorisation:

```
* [0, 5.2]     - Very Low
* (5.2, 11.2]  - Low
* (11.2, 21.3] - Moderate
* (21.3, 38.0] - High
* (38.0, 50.0] - Very High
* (50.0, Inf)  - Extreme
```

As seen, the deepFWI predictions are consistent with FWI-Forecast and even offer better accuracy for longer term predictions in certain FWI categories. 

In [6]:
# Generate result plots
!python src/plot.py -f result.json -i 4 -o 14
!ls acc* mae* mse*

acc_20201213-064623.png  mae_20201213-064631.png  mse_20201213-064627.png


In [7]:
Image(filename='acc_20201213-064623.png')